# Generating a 4D-Communication Tensor from computed communication scores

After inferring communication scores for combinations of ligand-receptor and sender-receiver cell pairs, we can use that information to identify context-dependent CCC patterns across multiple samples simultaneously by generating a 4D-Communication Tensor. LIANA handily outputs these score as a dataframe that is easy to use for building our tensor.

In this tutorial we will show you how to use the dataframe saved from LIANA to generate a 4D-Communication Tensor that could be later used with Tensor-cell2cell.

## Initial Setup

**Before running this notebook** 

GPUs can substantially speed-up the downstream decomposition. If you are planning to use a NVIDIA GPU, make sure to have a proper NVIDIA GPU driver (https://www.nvidia.com/Download/index.aspx) as well as the CUDA toolkit (https://developer.nvidia.com/cuda-toolkit) installed.

Then, make sure to create an environment with PyTorch version >= v1.8.1 following these instructions to enable CUDA.

https://pytorch.org/get-started/locally/

### Enabling GPU use

If you are using a NVIDIA GPU, after installing PyTorch with CUDA enabled, specify `gpu_use = True`. Otherwise, `gpu_use = False`. In R, this must be specified during tensor building. 

In [16]:
gpu_use = TRUE

if (gpu_use){
    device<-'cuda:0'
    tensorly <- reticulate::import('tensorly')
    tensorly$set_backend('pytorch')
}else{
    device<-NULL
}

**First, load the necessary libraries**

In [1]:
library(liana, quietly = T)
library(reticulate, quietly = T)

In [2]:
data_folder = '/data/hratch/ccc_protocols/interim/liana-outputs/'#'../../data/liana-outputs/'  # <--replace in final version
output_folder = '../../data/tc2c-outputs/'

env_name = 'ccc_protocols' # conda environemnt created by ../../env_setup/setup_env.sh

## Load Data

Open the list containing LIANA results for each sample/context. These results contain the communication scores of the combinations of ligand-receptor pairs and sender-receiver pairs.

In [4]:
context_df_dict<-readRDS(paste0(data_folder, 'LIANA_by_sample_R.rds'))

## Create 4D-Communication Tensor

### Specify the order of the samples/contexts

Here, we will specify an order of the samples/contexts given the condition they belong to (HC or *Control*, M or *Moderate COVID-19*, S or *Severe COVID-19*).

In [5]:
sorted_names = sort(names(context_df_dict))

In [6]:
sorted_names

[1] "HC1" "HC2" "HC3" "M1"  "M2"  "M3"  "S1"  "S2"  "S3"  "S4"  "S5"  "S6"

In [7]:
head(context_df_dict$HC1)

source,target,ligand.complex,receptor.complex,LRscore,sample_new
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
T,NK,B2M,CD3D,7.051711e-12,HC1
Macrophages,NK,B2M,CD3D,5.641369e-11,HC1
NK,NK,B2M,CD3D,8.814638e-10,HC1
T,NK,B2M,KLRD1,2.418737e-09,HC1
B,NK,B2M,CD3D,3.610476e-09,HC1
Macrophages,NK,B2M,KLRD1,3.610476e-09,HC1


## Generate tensor

To generate the 4D-communication tensor, we will to create matrices with the communication scores for each of the ligand-receptor pairs within the same sample, then generate a 3D tensor for each sample, and finally concatenate them to form the 4D tensor.

Briefly, we use the LIANA dataframe and communication scores to organize them as follows:

![ccc-scores](https://github.com/earmingol/cell2cell/blob/master/docs/tutorials/ASD/figures/4d-tensor.png?raw=true)

Score pre-processing:

In the previous [tutorial](./02-Infer-Communication-Scores.ipynb), we calculated the aggregate magnitude rank as the communication score. This score considers low values as the most important ones, ranging from 0 to 1. However, Tensor-cell2cell requires higher values to be the most important scores. So, prior to running Tensor-cell2cell, we subtract the rank score from 1 to adapt it. 

Additionally, Tensor-cell2cell expects non-negative scores. If you used a pipeline that generated negative scores, we suggest replacing these either with 0. Otherwise, by default, Tensor-cell2cell will treat these as NaN. Since we used the magnitude rank score, which is non-negative, that line of code will not change our scores.

Finally, since we will be setting `how` = 'outer' with `fill` set to 0, to decrease the number of indices the decomposition must disregard, we will drop cells or LRs that are not present in atleast 1/3 of the samples. 

<span style='color:red'>**These are the parameters in python missing from R, wrote a function in case want to adapt and incorporate**</span>

In [8]:
preprocess.scores<-function(context_df_dict, 
                            score_col = "LRscore", 
                            sender_col = "source", receiver_col = "target", 
                            ligand_col = "ligand.complex", receptor_col = "receptor.complex", 
                            outer_fraction = 0, invert = TRUE, non_negative = TRUE, non_negative_fill = 0
                           ){
    source.cells<-sapply(context_df_dict, function(x) x[[sender_col]])
    target.cells<-sapply(context_df_dict, function(x) x[[receiver_col]])
    all.cells.persample<-sapply(names(source.cells), 
                                function(sample.name) union(source.cells[[sample.name]], target.cells[[sample.name]]))
    all.cells<-Reduce(union, unname(unlist(all.cells.persample)))
    cell.counts.persample<- sapply(all.cells, 
                                   function(ct) length(which(sapply(all.cells.persample, function(x) ct %in% x))))
    cells.todrop<-names(which(cell.counts.persample < (outer_fraction*length(context_df_dict))))


    lrs.persample<-sapply(context_df_dict, function(x) paste(x[[ligand_col]], x[[receptor_col]], 
                                                             sep = '^'))
    all.lrs<- Reduce(union, unname(unlist(lrs.persample)))
    lr.counts.persample<-sapply(all.lrs, function (lr) length(which(sapply(lrs.persample, function(x) lr %in% x))))
    lrs.todrop<-names(which(lr.counts.persample < (outer_fraction*length(context_df_dict))))

    for (sample.name in names(context_df_dict)){
        ccc.sample<-context_df_dict[[sample.name]]

        if (invert){ccc.sample[[score_col]]=(1- ccc.sample[[score_col]])}# invert score
        if (non_negative){ccc.sample[[score_col]][ccc.sample[[score_col]] < 0] = non_negative_fill} # make non-negative

        # apply the outer_frac parameter
        ccc.sample[(!(ccc.sample[[sender_col]] %in% cells.todrop)) &
                   (!(ccc.sample[[receiver_col]] %in% cells.todrop)),]
        ccc.sample<-ccc.sample[!(paste(ccc.sample[[ligand_col]], ccc.sample[[receptor_col]], 
                                   sep = '^') %in% lrs.todrop),]

        context_df_dict[[sample.name]]<-ccc.sample
    }
    return(context_df_dict)
}

In [9]:
context_df_dict<-preprocess.scores(context_df_dict, outer_frac = 1/3, 
                                   invert = TRUE, non_negative = TRUE, non_negative_fill = 0)

**The key parameters when building a tensor are:**

- `how` controls what ligand-receptor pairs and cell types to include when building the tensor. This decision is dependent on the number of samples including scores for their combinations of ligand-receptor and sender-receiver cell pairs. Options are:
    - `'inner'` is the more strict option since it only considers only cell types and LR pairs that are present in all contexts (intersection).
    - `'outer'` considers all cell types and LR pairs that are present across contexts (union).
    - `'outer_lrs'` considers only cell types that are present in all contexts (intersection), while all LR pairs that are present across contexts (union).
    - `'outer_cells'` considers only LR pairs that are present in all contexts (intersection), while all cell types that are present across contexts (union).

- `fill` indicates what value to assign missing scores when `how` is set to `'outer'`, i.e., those cells or LR pairs that are not present in all contexts. During tensor component analysis, NaN values are masked such that they are not considered by the decomposition objective. This results in behavior of NaNs being imputed as missing values that are potentially communicating, whereas if missing LRs are filled with a value such as 0, they are treated as biological zeroes (i.e., not communicating). For additional details and discussion regarding this parameter, please see the [missing indices benchmarking](../../tc2c_benchmark/scripts/missing_indices_consistency.ipynb).
    - `'lf_fill'`: value to fill missing (across contexts) LR pairs with
    - `'cell_fill'`: value to fill missing (across contexts OR LR pairs) cell types with

**Commented out the arguments that are available in li.multi.to_tensor_c2c but not liana::liana_tensor_c2c**

In [10]:
tensor <- liana_tensor_c2c(context_df_dict = context_df_dict,
                          sender_col = "source", # Column name of the sender cells
                          receiver_col = "target", # Column name of the receiver cells
                          ligand_col = "ligand.complex", # Column name of the ligands
                          receptor_col = "receptor.complex", # Column name of the receptors
                        score_col = 'LRscore', # Column name of the communication scores to use
#                         inverse_fun=lambda x: 1 - x, # Transformation function
#                         non_expressed_fill=None, # Value to replace missing values with 
                        how='outer',  # What to include across all samples
                        lr_fill = NaN, # What to fill missing LRs with 
                        cell_fill = NaN, # What to fill missing cell types with 
#                         outer_fraction=1/3., # Fraction of samples as threshold to include cells and LR pairs.
                        lr_sep='^', # How to separate ligand and receptor names to name LR pair
                        context_order=sorted_names, # Order to store the contexts in the tensor
                        sort_elements = TRUE, # Whether sorting alphabetically element names of each tensor dim. Does not apply for context order if context_order is passed.
                        conda_env = env_name, # used to pass an existing conda env with cell2cell
                        build_only = TRUE, # set this to FALSE to combine the downstream rank selection and decomposition steps all here 
                        device = device # Device to use when backend is pytorch.
                            )

[1] 0


Loading `ccc_protocols` Conda Environment

Building the tensor...



## Evaluate some tensor properties

### Tensor shape
This indicates the number of elements in each tensor dimension: (Contexts, LR pairs, Sender cells, Receiver cells)

In [11]:
dim(tensor$tensor)

NULL

### Missing values
This represents the fraction of values that are missing. In this case, missing values are combinations of contexts x LR pairs x Sender cells x Receiver cells that did not have a communication score or were missing in the dataframes.

In [12]:
tensor$missing_fraction()

[1] 0.8899442

### Sparsity
This represents the fraction of values that are a real zero (excluding the missing values)

In [13]:
tensor$sparsity_fraction()

[1] 0.03689805

### Fraction of excluded elements
This represents the fraction of values that are ignored (masked) in the analysis. In this case it coincides with the missing values because we did not generate a new `tensor.mask` to manually ignore specific values. Instead, it automatically excluded the missing values.

In [14]:
tensor$excluded_value_fraction() # Percentage of values in the tensor that are masked/missing

[1] 0.8899442

## Export Tensor

Here we will save the `tensor` so we can use it later with other analyses.

In [15]:
reticulate::py_save_object(object = tensor, 
                           filename = paste0(output_folder, 'BALF-Tensor-R.pkl'))